In [ ]:
import copy
import time
from cmath import sin
import matplotlib.pyplot as plt

In [ ]:
def create_matrix_A(N, a1, a2, a3):
    A = [[0 for _ in range(N)] for _ in range(N)]
    b = [sin(i * (f + 1)) for i in range(N)]

    for i in range(N):
        for j in range(N):
            if i == j:
                A[i][j] = a1
            elif i == j + 1 or i == j - 1:
                A[i][j] = a2
            elif i == j + 2 or i == j - 2:
                A[i][j] = a3
    return A, b

In [ ]:
def jacobian_method(A, b, max_itr=1000, tolerance=1E-9):
    N = len(b)
    x = [1 for _ in range(N)]
    itr = 0
    residuals = []
    start = time.time()

    while itr < max_itr:
        x_new = [1 for _ in range(N)]
        for i in range(N):
            sum_left = 0
            for j in range(i):
                sum_left += A[i][j] * x[j]
            sum_right = 0
            for j in range(i + 1, N):
                sum_right += A[i][j] * x[j]

            x_new[i] = (b[i] - sum_left - sum_right) / A[i][i]

        residual = [0 for _ in range(N)]
        err_norm = float('-inf')

        for i in range(N):
            sum_Ax = 0
            for j in range(N):
                sum_Ax += A[i][j] * x[j]
            residual[i] = sum_Ax - b[i]
            if abs(residual[i]) > err_norm:
                err_norm = abs(residual[i])

        if err_norm < tolerance:
            break

        x = x_new
        residuals.append(err_norm)
        itr += 1

    return itr, residuals, x, time.time() - start

In [ ]:
def gauss_seidel_method(A, b, max_itr=1000, tolerance=1E-9):
    N = len(b)
    x = [1 for _ in range(N)]
    itr = 0
    residuals = []
    err_norm = None
    start = time.time()

    while itr < max_itr:
        x_new = [1 for _ in range(N)]
        for i in range(N):
            sum_left = 0
            for j in range(i):
                sum_left += A[i][j] * x_new[j]
            sum_right = 0
            for j in range(i + 1, N):
                sum_right += A[i][j] * x[j]

            x_new[i] = (b[i] - sum_left - sum_right) / A[i][i]

        residual = [0 for _ in range(N)]
        err_norm = float('-inf')

        for i in range(N):
            sum_Ax = 0
            for j in range(N):
                sum_Ax += A[i][j] * x[j]
            residual[i] = sum_Ax - b[i]
            if abs(residual[i]) > err_norm:
                err_norm = abs(residual[i])

        if err_norm < tolerance:
            break

        x = x_new
        residuals.append(err_norm)
        itr += 1

    return itr, residuals, x, time.time() - start

In [ ]:
def LU_factorization(A, b):
    U = copy.deepcopy(A)
    L = [[0 for _ in range(N)] for _ in range(N)]

    start = time.time()

    for i in range(N):
        for j in range(i):
            L[i][j] = U[i][j] / U[j][j]
            for k in range(j, N):
                U[i][k] -= L[i][j] * U[j][k]

    # Solve Ly = b
    y = [0 for _ in range(N)]
    for i in range(N):
        y[i] = b[i]
        for j in range(i):
            y[i] -= L[i][j] * y[j]

    # Solve Ux = y
    x = [0 for _ in range(N)]
    for i in range(N - 1, -1, -1):
        x[i] = y[i]
        for j in range(i + 1, N):
            x[i] -= U[i][j] * x[j]
        x[i] /= U[i][i]

    # Calculate residual norm
    residual = [0 for _ in range(N)]
    err_norm = float('-inf')

    for i in range(N):
        sum_Ax = 0
        for j in range(N):
            sum_Ax += A[i][j] * x[j]
        residual[i] = sum_Ax - b[i]
        if abs(residual[i]) > err_norm:
            err_norm = abs(residual[i])

    return err_norm, x, time.time() - start

In [ ]:
# Initial values
index_number = 193320
c = 1
d = 2
e = 3
f = 3
N = 9 * c * d
a1 = 5 + e
a2 = -1
a3 = -1

In [ ]:
N_values = [100, 500, 1000, 2000, 3000]
time_jacobie_list = []
time_seidel_list = []
time_LU_list = []

In [ ]:
for N in N_values:
    A, b = create_matrix_A(N, a1, a2, a3)
    _, _, _, time_jacobie = jacobian_method(A, b)
    time_jacobie_list.append(time_jacobie)
    _, _, _, time_seidel = gauss_seidel_method(A, b)
    time_seidel_list.append(time_seidel)
    _, _, time_LU = LU_factorization(A, b)
    time_LU_list.append(time_LU)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(N_values, time_jacobie_list, marker='o', label='Metoda Jacobiego')
plt.plot(N_values, time_seidel_list, marker='s', label='Metoda Gaussa-Seidla')
plt.plot(N_values, time_LU_list, marker='^', label='Metoda faktoryzacji LU')
plt.xlabel('Liczba niewiadomych (N)')
plt.ylabel('Czas wyznaczenia rozwiązania (s)')
plt.title('Zależność czasu wyznaczenia rozwiązania od liczby niewiadomych')
plt.legend()
plt.grid(True)
plt.show()